# Speaker and data

You speaker (or the one you want to buy) has some properties: size, how loud they can go, do they have an amplifier, etc?

A U.S. standard has defined a series of graphs called a spinorama that help to characterize your speaker. It will answer most of your question about this speaker. It will tell nothing about it's size, intended usage, design or price.

# How to read a spinorama graphs?

this is a *gentle* tutorial that tries to explain all graphs on a *spinorama* and why they are important.

In [2]:
from math import log, sqrt
import numpy as np
import pandas as pd
import altair as alt

In [3]:
def db(f):
    if f>10000:
        return 87-f/5000
    elif f>120:
        return 85
    elif f>20:
        return 85-sqrt(120-20)*2 + sqrt(f-20)*2
    elif f>0:
        return (f/2)+25
    else:
        return 0
x = np.logspace(1, 4+log(2), 60)
y = [db(x[i]) for i in range(0, len(x))]
yp = [85 for i in range(0, len(x))]
yr = [iy+(np.random.rand()-0.5)*6 for iy in y]

In [4]:
fx = pd.DataFrame({'freq': x})
f  = pd.DataFrame({'freq': x, 'dB': y})
fp = pd.DataFrame({'freq': x, 'dB': yp})
fr = pd.DataFrame({'freq': x, 'dB': yr})

## Let's start with a frequency graph.

### Frequency? 
Sound is carried by waves in the atmosphere. It oscillates at a frequency
measured in Hertz (Hz).

A low number means bass, high number means treble. People can usually hear 
between 16 hz and 20000 Hz. This varies with people and usually reduce 
significantly as you get older. Speech is around 1000 hz. I also use k 
for kilo Hz thus 10000Hz equals 10kHz.

### Decibel or dB?
To make it simple, think about volume, more decibel means higher volume 
(sound pressure). Note that:

- +3dB double the volume because it is a logarithmic scale. 
- -3dB halves the volume.

In practice, your ears do not react in the same way for bass, midrange and for treble
and this is not completly true. This a realative value (value itself is 
meaningless but differences make sense).

### Frequency graph
The frequency graph represent the sound pressure *aka* volume (in dB) at a given frequency in Hz. 
They are usually represented with the frequency on a logarithmic scale too in order to see better 
the important information between 20hz and 500hz.

Below we have an example of a speaker (the contious line). It is flat from 120hz to 10000hz in this example. Below 120hz, it drops significantly. After 10000hz, it also drops a bit.
For the bass area, you need to move a lot of air to produce bass at high volume. The speaker needs to be large to produce a lot of bass but even a large speaker cannot produce that much bass. For high frequencies, the tweeter also usually dip a bit at high frequency but this is not always the case.

A perfect speaker would be a flat line (here with dash) but this is physically impossible.
A good speaker will be as close as possible to the flat line.

Note that we do not care what happens after 20k hz because we do not hear it (your dog or a bat may). Similarly we don't care about below 16hz. Music is mostly above 40hz. Electronic music and movies goes easily down to 20hz. You will need a dedicated speaker called a subwoofer to reproduce this low frequencies well and at high volume.

In [5]:
a_x = alt.X('freq:Q', scale=alt.Scale(type='log', nice=False, domain=[20, 20000]))
a_y = alt.Y('dB:Q', scale=alt.Scale(type='linear', domain=[45, 90]))
g_f  = alt.Chart(f).mark_line(clip=True).encode(x=a_x, y=a_y)
g_fp = alt.Chart(fp).mark_line(clip=True, strokeDash=[4,2]).encode(x=a_x, y=a_y)
g_f+g_fp

alt.LayerChart(...)

## Theory and reality

Theory is nice, but how does that looks like in practice?
Reality is as often not as nice and the curve of a measured speaker is a lot more wobby.
Here is an example of what it can looks like. Note: this is not real data yet, i made this one up,
but it gives an idea.

In [6]:
g_fr = alt.Chart(fr).mark_line(clip=True, strokeDash=[2,2], color='black').encode(x=a_x, y=a_y)
g_f+g_fr

alt.LayerChart(...)

## How bad or how good is that?

The purpose of measurement is to have *data* to evaluate how far or how close your speaker is to a perfect one. 

The *bad* news is : it is *hard* to get accurate data.
The *good* news is : it is *easy to understand*.

Why it is hard to get data? Manufacturer don't provide them. But why if they are easy to understand? 

I don't know for sure. Maybe the marketing from this company prefers to focus you on the look or on whatever parameter
they think will decide you to buy. Since price and sound quality do not correlate well, that's a likely possibility. 
Telling: "my 10k speaker looka better but sounds the same as my 1k speaker" is not very appealing for them. 

The data will let you decide on this part.

## Wobbiness or deviation from flat

One red line is at the maximim of frequency over a range (here from 100hz to 10000hz).
The other line is at the minimum.

The amplitude between the 2 lines is giving an idea on how your speaker is deviating from the theortical perfect flat line.

You will see specification like: +/- 3dB between 100hz and 10000hz.
Is that good? +/- 3dB is a doubling or halving of the volume, that's already a lot of differences between some part of the frequency range and another.

Best speakers in the world are able to do +/-1 dB over that range. Let's say that below +/- 2dB this is very good.

In [7]:
line_up = alt.Chart(fx).mark_line(color='red').encode(x='freq:Q', y='dB:Q'
        ).transform_calculate(dB='88')
line_down = alt.Chart(fx).mark_line(color='red').encode(x='freq:Q', y='dB:Q'
        ).transform_calculate(dB='82')
g_fr+line_up+line_down

alt.LayerChart(...)

## How much bass?

The second important parameter is how low can you speaker go?
Let's add a green line which is a best fit of the speaker curve.
Best fit means linear regression for the curious.

In [8]:
reg = g_fr.transform_filter(
    'datum.freq>120 & datum.freq<10000'
).transform_regression('freq', 'dB', method='log', extent=[20, 20000]
).mark_line(color='green')
g_fr + reg

alt.LayerChart(...)

Let's add a red line -6dB below the best fit curve.

In [9]:
reg_down = reg.transform_calculate(dB='datum.dB-6').mark_line(color='red')
g_fr+reg+reg_down

alt.LayerChart(...)

In [10]:
lfx = 74
intersec = {'freq': [lfx,], 'dB': [78, ], 'textof': ['➟',]}
d_intersec = pd.DataFrame(intersec)

d_point = alt.Chart(d_intersec).mark_point(size=50).encode(x='freq', y='dB')
d_text  = alt.Chart(d_intersec).mark_text(
    dx=-20,dy=0, angle=45, fontSize=35).encode(x='freq',y='dB', text='textof')

## Low Frequency Extension (LFX)

The intersection between the frequency graph and the red line is called LFX (Low Frequency Extension). You will see it in specification: 50hz at -6dB (sometimes -10dB or no indication).

### What is good what is bad?
Here good and bad are a function of the physical size of your speaker. If it is a bookshelve, 
50hz is good, it is a large tower, you can go as low as 30hz and sometimes less.

You will see that the price climbs steadily when LFX goes down. It is usually cheaper to add 
a subwoofer (or two) to a pair of bookshelves than to buy large towers.

Bass are nice! Your neibourgh may disagree.

### Why -6dB as a measuring point?

All measurement are supposed to be done in a special anechoic room.
In your room, you have walls that will reflect the sound and increase the volume.
If your speakers are against a wall or in a corner, the volume will increase by 6dB (resp 12dB). 
That's also why you will hear too much bass if you put your speaker in a corner.

### Low and high volume and perceived amount of bass?

Your ears are also complicated. Let's suppose you are in your sofa listening to music. 
If the volume is low, you need more bass and more trebel. If the volume increases a lot, 
then you don't need to boost bass and treble. We used to have settings on amplifiers to 
compensate for this ([loudness compensation](https://en.wikipedia.org/wiki/Loudness_compensation)).




In [11]:
g_fr+reg+reg_down + d_point + d_text

alt.LayerChart(...)

# A real speaker

Let's look at the same graphs but with real datas this time. I picked up a *very* good one.

## On Axis 

This is very well behaving speaker. It is very flat on axis: 

- deviation 45Hz-20kHz: constructor (+/- 1.5 dB), independantly measured (+/- 1.4 dB)
- Low Frequency Extension: spec (38Hz at -6dB), independatly measured (34Hz at -6dB)

The constructor has excellent specs and even beat them on this model.


In [12]:
from src.spinorama.analysis import sound_power
from src.spinorama.normalize import normalize_cea2034, normalize_mean
from src.spinorama.load import parse_all_speakers, parse_graphs_speaker, graph_melt
from src.spinorama.graph import graph_params_default, graph_freq, graph_spinorama

In [13]:
genelec = parse_graphs_speaker('Genelec', 'Genelec 8341A', 'klippel')
genelec_cea2034 = genelec['CEA2034']

cea2034 = normalize_cea2034(genelec_cea2034, normalize_mean(genelec_cea2034))
onaxis = cea2034[cea2034.Measurements == 'On Axis']

graph_spinorama(onaxis, graph_params_default)

alt.LayerChart(...)

## What's happening in your room?

Let's suppose you have your speakers in a normal rectangular room. This is such a common use case that the standard has been build around this case.
The On axis graph we have been looking at before will be changed by your room into a different graph. This is the Estimated In-Room Response graph:

In [14]:
from src.spinorama.display import display_inroom
display_inroom(genelec)

alt.LayerChart(...)

This is again perfect behaviour: a slopping down response with very little deviation from the straight red line.

## What about reflections?

The sound comes from the speaker and you get hit by the soudwave. At the same time, the speaker is radiating sound not only at you but in many directions (more or less in all directions, obviously less behind and on the side than in front but still a lot). This waves sound will hit walls and bounce. The bounce depend a lot of what the surface is: the harder the surface, the bigger the reflections. One of the reason, people put a big carpet in front of the speaker is to reduce the bounce from the ground.

## What's happening off axis?

Let's measure the speaker response at an angle from the on axis. The standard ask for 2 loops: one horizontal in 10 degre increments, one vertical in 10 degre increments.
You have 360/10=36 graphs for the horizontal plane and the same for the vertical plane. 

Let's plot some of them (from on-axis to 60 degre) both on the vertical and horinzontal plane. 

In [15]:
from src.spinorama.display import display_spl_horizontal, display_spl_vertical
splH = display_spl_horizontal(genelec)
splV = display_spl_vertical(genelec)
splV | splH

alt.HConcatChart(...)

Here we see why the speaker is very good: if you are off axis by 10, 20 or 30 degrees you are still getting the same sound as if you were perfectly align with the speaker axis.
This is very uncommon most speakers exibit large differences between on axis and slightly off measurements. 

## What's a contour plot?

A contour plot is the same information has above but presented in a different way: we still have frequency in abscisse but this time we have the angle on the vertical axis.
The color model the difference from on axis. The scale is plotted on the left. Basically red and shades off means less than 3dB difference from on axis. 

In [18]:
from src.spinorama.display import display_contour_horizontal, display_contour_vertical
contourH = display_contour_horizontal(genelec)
contourV = display_contour_vertical(genelec)
contourV | contourH

alt.HConcatChart(...)

## What's a radar plot?

A radar plot is another way to plot the same data, this time focusing on the result at some specific frequencies.


In [16]:
from src.spinorama.display import display_radar_horizontal, display_radar_vertical
radarH = display_radar_horizontal(genelec)
radarV = display_radar_vertical(genelec)
radarV | radarH

alt.HConcatChart(...)

## Spinorama 

## How would a less good speaker looks like?

